In [6]:
# auto-encoder example 0-9 digit
# 1 import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

import torchvision.datasets as datasets
import torchvision.transforms as transforms

import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# prepare dataset
transforms = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(
    root="data2", train=True, transform=transforms, download=True
)

test_dataset = datasets.MNIST(
    root="data2", train=False, transform=transforms, download=True
)

train_loder = DataLoader(
    dataset=train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=False
)

test_loader = DataLoader(
    dataset=test_dataset, batch_size=32, shuffle=False, num_workers=4
)

100%|██████████| 9912422/9912422 [00:24<00:00, 405075.63it/s] 


Extracting data2\MNIST\raw\train-images-idx3-ubyte.gz to data2\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 1697791.05it/s]

Extracting data2\MNIST\raw\train-labels-idx1-ubyte.gz to data2\MNIST\raw



100%|██████████| 1648877/1648877 [00:03<00:00, 493211.45it/s]


Extracting data2\MNIST\raw\t10k-images-idx3-ubyte.gz to data2\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data2\MNIST\raw\t10k-labels-idx1-ubyte.gz to data2\MNIST\raw



In [3]:
# 3 Network Encoder
class Encoder(nn.Module):
    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()

        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=0),
            nn.ReLU(True),
        )

        self.flatten = nn.Flatten(start_dim=1)
        self.encoder_lin = nn.Sequential(
            nn.Linear(3 * 3 * 32, 128), nn.ReLU(True), nn.Linear(128, encoded_space_dim)
        )

    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x


class Decoder(nn.Module):
    def __init__(self, decoded_space_dim, fc2_input_dim):
        super().__init__()
        self.decoded_lin = nn.Sequential(
            nn.Linear(decoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 3 * 3 * 32),
            nn.ReLU(True),
        )

        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, 3, 3))
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU6(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1),
        )

    def forward(self, x):
        x = self.decoded_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [4]:
# 4 loss function optimizer
encode = Encoder(encoded_space_dim=4, fc2_input_dim=128).to(device)
decoder = Decoder(decoded_space_dim=4, fc2_input_dim=128).to(device)

params_to_optimize = [
    {"params": encode.parameters()},
    {"params": decoder.parameters()},
]
optim = torch.optim.Adam(params_to_optimize, lr=0.001, weight_decay=1e-05)
loss_fn = torch.nn.MSELoss()

In [5]:
# 5 add noise
def add_noise(inputs, noise_factor=0.3):
    noisy = inputs + torch.randn_like(inputs) * noise_factor
    noisy = torch.clip(noisy, 0.0, 1.0)
    return noisy

In [7]:
# 6 train function
def train_epoch(
    encoder, decoder, device, dataloader, loss_fn, optimizer, noise_factor=0.3
):
    encoder.train()
    decoder.train()
    train_loss = []

    for image_batch, _ in dataloader:
        image_noisy = add_noise(image_batch, noise_factor)
        image_noisy = image_noisy.to(device)
        encoded_data = encoder(image_noisy)
        decoded_data = decoder(encoded_data)
        loss = loss_fn(decoded_data, image_noisy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())
    return np.mean(train_loss)